Sam Ellis

## 0) Prelims

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
import torch

In [3]:
import torchvision.models as models
resnet18 = models.resnet18(pretrained=True)
alexnet = models.alexnet(pretrained=True)
squeezenet = models.squeezenet1_0(pretrained=True)
vgg16 = models.vgg16(pretrained=True)
densenet = models.densenet161(pretrained=True)
inception = models.inception_v3(pretrained=True)
googlenet = models.googlenet(pretrained=True)
shufflenet = models.shufflenet_v2_x1_0(pretrained=True)
mobilenet = models.mobilenet_v2(pretrained=True)
resnext50_32x4d = models.resnext50_32x4d(pretrained=True)
wide_resnet50_2 = models.wide_resnet50_2(pretrained=True)
mnasnet = models.mnasnet1_0(pretrained=True)

models = [('googlenet', googlenet, []),('shufflenet', shufflenet, []),('mobilenet', mobilenet, []),('resnext50_32x4d', resnext50_32x4d, []),
          ('wide_resnet50_2', wide_resnet50_2, []),('mnasnet', mnasnet, []), ('resnet18', resnet18, []),('alexnet', alexnet, []),
          ('squeezenet', squeezenet, []),('vgg16', vgg16, []),('densenet', densenet, [])]

If youve read my ResNets notebook, the same applies here. These variables need to point to the right places. 

In [4]:
im_path = '/elpv-dataset'

train_path = 'train_csv'
test_path = 'test_csv'

print(len(models))

11


## 1) Dataset

This is quite similar to the dataset/loader in my ResNet notebook, however this time we stack the image on itself 3 times. This is a haphazard way of pumping grayscale images through CNN's trained on RBG images. 

In [5]:
import pandas as pd
import torch
import os
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from torchvision.transforms import ToTensor
import math
import torch.optim as optim
import time

class solar_cell_dataset(Dataset):
    def __init__(self, csv_file, img_dir, transform=None, rgb = True):
        self.labels = pd.read_csv(csv_file)
        self.img_dir = img_dir
        self.transform = transform
        self.rbg = rgb

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = os.path.join(self.img_dir, self.labels.iloc[idx, 0])
        img = Image.open(img_name)

        target = torch.tensor(self.labels.iloc[idx, 1:])
        target = target.type(torch.FloatTensor)

        if self.transform:
          img = self.transform(img)
        if self.rbg:
          img = img.repeat(3, 1, 1)

        return img, target

##  2) FC Linear Transfer Learner

Very simple class below. Just a two layer linear network with a ReLu.

In [6]:
import torch.nn as nn
class transfer_learner(nn.Module):
  def __init__(self):
    super(transfer_learner, self).__init__()
    self.fc1 = nn.Linear(1000, 1000)
    self.fc2 = nn.Linear(1000, 2)
    self.relu = nn.ReLU()
    self.soft = nn.Softmax(dim=1)

  def forward(self, x):
    x = self.relu(self.fc1(x))
    x = self.fc2(x)
    return x

##  3) Loading Models and Data

In [7]:
######### HYPER PARAMS ###########
batch_size = 64
cores = 0 #-1
lr = .001
momentum = .6
epochs = 100
################################
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [8]:
from torchvision.transforms.transforms import RandomApply
import torchvision.transforms as T

transforms = T.Compose([
    T.ToTensor(),
    T.RandomApply(nn.ModuleList([T.RandomAffine(degrees = (0, 30), translate = (0.1, 0.2), scale = (0.5, 1))]), p=.2)
])

s = solar_cell_dataset(csv_file=train_path, img_dir=im_path, transform=ToTensor())
t = solar_cell_dataset(csv_file=test_path, img_dir=im_path, transform=ToTensor())

train_loader = DataLoader(s,
                          batch_size=batch_size,
                          num_workers=cores)
test_loader = DataLoader(t,
                         batch_size=batch_size,
                         num_workers=cores)

In [9]:
print(s.__getitem__(0)[0].shape)

torch.Size([3, 300, 300])


In [10]:
import torch.nn.functional as F
s_ = nn.Softmax(dim=1)
def classify(x):
  x = s_(x)
  x = torch.argmax(x, dim=1)
  return x

The training loop works like this: get a model, stack the neural net on top of it, freeze the gradient of the cnn, train the NN and test every 9 epochs. During all this I record the best test accuracy. To generate the table in our report, I ran this twice. 

The first time I transformed the training data with ToTensor(), and the second time I used my transforms() above which applies a random affine transormation with probability .2.

In [11]:
loss_criterion = nn.CrossEntropyLoss()

for name, model, test_acc in models:
  for param in model.parameters():
    param.requires_grad = False
  transfer = transfer_learner()

  net = nn.Sequential(model, transfer)
  net.to(device)
  optimizer = optim.SGD(net.parameters(), lr = lr, momentum = momentum)

  for epoch in range(epochs):
    t_loss = 0
    acc = 0
    for idx, data in enumerate(train_loader):
      img, target = data
      img, target = img.to(device), target.to(device)

      optimizer.zero_grad()

      output = net(img)
      loss = loss_criterion(output, target)
      loss.backward()
      optimizer.step()

      t_loss += loss.item()
      acc += (classify(output) == torch.argmax(target, dim=1)).float().sum()
    accuracy = 100 * acc / (len(s))
    print(f'({name}), [{epoch}], loss : {t_loss:.3f}, accuracy : {accuracy:.3f}%')

    if (epoch + 1) % 9 == 0:
      acc = 0
      for idx, data in enumerate(test_loader):
        img, target = data
        img, target = img.to(device), target.to(device)

        output = net(img)
        acc += (classify(output) == torch.argmax(target, dim=1)).float().sum()
      accuracy = 100 * acc / (len(t))
      test_acc.append(accuracy)   
      print('======================================================')
      print(f'[{epoch}], TEST accuracy : {accuracy:.3f}%')
      print('======================================================')
  print(torch.cuda.memory_allocated())
  del net
  print(torch.cuda.memory_allocated())

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:780: UserWarning: Note that order of the arguments: ceil_mode and return_indices will changeto match the args list in nn.MaxPool2d in a future release.
  warnings.warn("Note that order of the arguments: ceil_mode and return_indices will change"


(googlenet), [0], loss : 18.051, accuracy : 61.474%
(googlenet), [1], loss : 17.919, accuracy : 66.817%
(googlenet), [2], loss : 17.760, accuracy : 67.885%
(googlenet), [3], loss : 17.576, accuracy : 68.391%
(googlenet), [4], loss : 17.360, accuracy : 69.010%
(googlenet), [5], loss : 17.158, accuracy : 69.460%
(googlenet), [6], loss : 16.938, accuracy : 69.966%
(googlenet), [7], loss : 16.868, accuracy : 70.360%
(googlenet), [8], loss : 16.636, accuracy : 70.247%
[8], TEST accuracy : 73.708%
(googlenet), [9], loss : 16.649, accuracy : 71.035%
(googlenet), [10], loss : 16.486, accuracy : 71.822%
(googlenet), [11], loss : 16.376, accuracy : 71.822%
(googlenet), [12], loss : 16.093, accuracy : 72.778%
(googlenet), [13], loss : 16.139, accuracy : 72.778%
(googlenet), [14], loss : 16.046, accuracy : 73.285%
(googlenet), [15], loss : 15.840, accuracy : 74.016%
(googlenet), [16], loss : 15.871, accuracy : 73.228%
(googlenet), [17], loss : 15.710, accuracy : 74.128%
[17], TEST accuracy : 77.75

In [12]:
important = pd.DataFrame(index = ['accuracy'])
for data in models:
  name, model, acc = data
  if acc:
    important[name] = max(acc).item()

print(important)

          googlenet  shufflenet  mobilenet  resnext50_32x4d  wide_resnet50_2  \
accuracy  84.494385   83.370789  82.471909        78.426964        78.202248   

            mnasnet  resnet18    alexnet  squeezenet      vgg16   densenet  
accuracy  76.853935  80.89888  78.651688   82.696632  83.370789  82.022476  
